In [1]:
import random
import numpy as np
import os
import sys
import torch
from torchvision.transforms import transforms
from torchvision import datasets
from collections import Counter
from torch import nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Subset, ConcatDataset, random_split
import numpy as np
from copy import deepcopy
import numpy as np
import torch
import torch.nn.functional as F
import pickle
import re
import os
import itertools
import csv

In [2]:
# Check if running on Colab
try:
  import google.colab
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  from google.colab import drive
  # Connect to Google drive where the training data is located
  drive.mount("/content/gdrive")
  work_dir = "/content/gdrive/My Drive/Colab Notebooks/DL-Project-2024-Experiments/SUBMISSION"
  os.chdir(work_dir)
  print(f"Connected to Google drive, setting working directory to '{work_dir}'")

Mounted at /content/gdrive
Connected to Google drive, setting working directory to '/content/gdrive/My Drive/Colab Notebooks/DL-Project-2024-Experiments/SUBMISSION'


In [3]:
# Create results folder if it does not exist yet
results_folder_name = 'task_1_task_2_final_accuracy_stats'
if not os.path.exists(results_folder_name):
    os.makedirs(results_folder_name)

In [4]:
def get_file_key(n_epochs, experiment, buffer_pct, goldilocks_rate, mpe_rate, goldilocks_remove_lowest_pct, goldilocks_remove_highest_pct, mpe_highest):
  return f'e2e_ep_{n_epochs}_ex_{experiment}_b_{buffer_pct}_gl_{goldilocks_rate}_mpe_{mpe_rate}_gll_{goldilocks_remove_lowest_pct}_glh_{goldilocks_remove_highest_pct}_mpeh_{mpe_highest}'

def reject(filter_map, value_map, key_list):
  for key in key_list:
    if (len(filter_map[key])>0 and value_map[key] not in filter_map[key]):
      return True
  return False

def parse_file_key(filename):
  pattern=r'task_accuracies_data_e2e_ep_(?P<n_epochs>\d+)_ex_(?P<experiment>\d+)_b_(?P<buffer_pct>[+-]?\d*\.\d+)_gl_(?P<goldilocks_rate>[+-]?\d*\.\d+)_mpe_(?P<mpe_rate>[+-]?\d*\.\d+)_gll_(?P<goldilocks_remove_lowest_pct>[+-]?\d*\.\d+)_glh_(?P<goldilocks_remove_highest_pct>[+-]?\d*\.\d+)_mpeh_(?P<mpe_highest>True|False).pkl'
  match = re.match(pattern, filename)
  if match:
    data = match.groupdict()
    #print(data)
    return data
  else:
    return None

all=[]
folder_paths=["raw_result_data"]
filters={"n_epochs": [50], "buffer_pct": [],"goldilocks_rate": [], "mpe_rate": [], "goldilocks_remove_lowest_pct": [], "goldilocks_remove_highest_pct": [],"mpe_highest": []}
for folder_path in folder_paths:
  for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      if os.path.isfile(file_path) and file_path.endswith('.pkl'):
          meta=parse_file_key(filename)
          loaded_results = None
          with open(file_path, 'rb') as f:
            loaded_results = pickle.load(f)
          if reject(filters, loaded_results, filters.keys()):
            continue
          group_key=get_file_key(loaded_results["n_epochs"], "X", loaded_results["buffer_pct"], loaded_results["goldilocks_rate"], loaded_results["mpe_rate"], loaded_results["goldilocks_remove_lowest_pct"], loaded_results["goldilocks_remove_highest_pct"], loaded_results["mpe_highest"])
          test_accuracies_1=loaded_results["test_accuracies_1"]
          test_accuracies_2=loaded_results["test_accuracies_2"]
          all.append((group_key, meta, test_accuracies_1, test_accuracies_2))

all.sort(key=lambda x: x[0])
grouped = itertools.groupby(all, key=lambda x: x[0])
grouped_dict = {key: list(group) for key, group in grouped}
print(f"{len(all)} files found, creating {len(grouped_dict.keys())} statistics")

stats=[]

for key, l in grouped_dict.items():
  n_epochs=int(l[0][1]['n_epochs'])
  buffer_pct=float(l[0][1]['buffer_pct'])
  goldilocks_rate=float(l[0][1]['goldilocks_rate'])
  mpe_rate=float(l[0][1]['mpe_rate'])
  goldilocks_remove_lowest_pct=float(l[0][1]['goldilocks_remove_lowest_pct'])
  goldilocks_remove_highest_pct=float(l[0][1]['goldilocks_remove_highest_pct'])
  mpe_highest=eval(l[0][1]['mpe_highest'])
  wildcard="X"
  epochs=np.arange(0,2*n_epochs)
  switch_epoch=n_epochs
  l1=[]
  l2=[]
  for i, series in enumerate(l):
    l1.append(l[i][2][-1])
    l2.append(l[i][3][-1])

  mean_1=np.mean(l1)
  mean_2=np.mean(l2)
  std_1=np.std(l1)
  std_2=np.std(l2)

  record=(n_epochs, buffer_pct, goldilocks_rate, mpe_rate, goldilocks_remove_lowest_pct, goldilocks_remove_highest_pct, mpe_highest, mean_1,std_1, mean_2, std_2 )
  stats.append(record)


370 files found, creating 37 statistics


In [5]:
header = ['n_epochs', 'buffer_pct', 'goldilocks_rate', 'mpe_rate', 'goldilocks_remove_lowest_pct', 'goldilocks_remove_highest_pct', 'mpe_highest', 'mean_1','std_1', 'mean_2', 'std_2']

csv_file=f'{results_folder_name}/task_1_task_2_final_accuracy_statistic.csv'

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(stats)

print(f"CSV file {csv_file} created successfully.")

CSV file task_1_task_2_final_accuracy_stats/task_1_task_2_final_accuracy_statistic.csv created successfully.
